# Kaggle Project

## Describe Your Dataset

**URL:** https://www.kaggle.com/datasets/samuelcortinhas/muffin-vs-chihuahua-image-classification/data

**Task:**
muffin과 chihuahua에 대한 구글 이미지 결과값을 스크랩한 바이너리 이미지 파일을 각각 구분하는 모델 만들기

**Datasets**

* Train dataset: train 파일 데이터의 70%

* Validation dataset: train 파일 데이터의 30%

* Test dataset: test 파일에 들어있는 이미지 전부

---

## Build Your Model

### Data preprocessing

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18

## gpu 사용 여부 확인 및 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## 데이터 경로 설정
train_data_path = "/Users/jung-yujin/2023DGU_MLDL_Final/train"
test_data_path = "/Users/jung-yujin/2023DGU_MLDL_Final/test"

## 데이터 전처리 및 augmentation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

## 데이터 로딩
dataset = ImageFolder(train_data_path, transform=transform)

## 데이터를 train과 validation으로 나누기
train_size = int(0.7 * len(dataset))
val_size = len(dataset) - train_size
train_data, val_data = random_split(dataset, [train_size, val_size])

## test 데이터 로딩
test_dataset = ImageFolder(test_data_path, transform=transform)

## 데이터 로더 설정
train_loader = DataLoader(train_data, batch_size=64, shuffle=True, num_workers=4)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

In [2]:
## 데이터 확인 (예시로 첫 번째 batch의 이미지와 레이블 출력)
for images, labels in train_loader:
    print(f"Image shape: {images.shape}, Label: {labels}")
    break

Image shape: torch.Size([64, 3, 224, 224]), Label: tensor([1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0,
        0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1,
        0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1])


### Model Construction

In [5]:
## 모델 정의 (ResNet18을 사용하고 출력 클래스를 2로 변경)
model = resnet18(weights=None)
model.fc = nn.Linear(model.fc.in_features, 2)  ## 출력 클래스에 맞게 변경

## 모델을 cpu로 이동
model = model.to(device)

### Train Model & Select Model

In [6]:
## 손실 함수 및 최적화 함수 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [7]:
## 모델 학습
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

## Validation Loss 계산
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            val_loss += criterion(outputs, labels).item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    accuracy = correct / total

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}, Validation Loss: {val_loss}, Accuracy: {accuracy}")

## 학습된 모델 저장
torch.save(model.state_dict(), "muffin_chihuahua_model.pth")

Epoch 1/10, Loss: 0.37075158953666687, Validation Loss: 1.6044188349143318, Accuracy: 0.6887323943661972
Epoch 2/10, Loss: 0.42714565992355347, Validation Loss: 1.4175522275592969, Accuracy: 0.793661971830986
Epoch 3/10, Loss: 0.36241209506988525, Validation Loss: 0.4926461201647054, Accuracy: 0.8091549295774648
Epoch 4/10, Loss: 0.2747235894203186, Validation Loss: 1.1277967497058536, Accuracy: 0.721830985915493
Epoch 5/10, Loss: 0.5478838086128235, Validation Loss: 0.4387102321438167, Accuracy: 0.8204225352112676
Epoch 6/10, Loss: 0.19325777888298035, Validation Loss: 0.3550372058930604, Accuracy: 0.8556338028169014
Epoch 7/10, Loss: 0.24344468116760254, Validation Loss: 0.28901107803634973, Accuracy: 0.8866197183098592
Epoch 8/10, Loss: 0.31232750415802, Validation Loss: 0.3742492218380389, Accuracy: 0.8591549295774648
Epoch 9/10, Loss: 0.29752498865127563, Validation Loss: 0.31825208016063855, Accuracy: 0.8767605633802817
Epoch 10/10, Loss: 0.26755717396736145, Validation Loss: 0.3

---

## Performance

In [8]:
## 모델 평가
model.eval()
test_correct = 0
test_total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = test_correct / test_total
print(f"Test Accuracy: {test_accuracy}")

Test Accuracy: 0.8851351351351351


The results explains

이미지 분류 모델은 '머핀 vs 치와와' 데이터셋에서 효과적으로 작동했습니다. 10 에포크 동안의 훈련 결과는 검증 정확도가 87.39%로 안정화되었으며, 테스트에서는 88.51%의 정확도를 달성했습니다. 초기에는 빠른 학습이 이루어졌고, 모델은 훈련 및 테스트 데이터에 대한 안정적인 성능을 보여주었습니다. 향후 개선을 위해 데이터셋의 특이성과 모델의 한계를 조사하는 것이 중요합니다